In [1]:
example = ('Navalnyj: Tvingas titta på tv i fängelset',
 'Ryske oppositionsledaren Aleksej Navalnyj – som hålls fängslad – tvingas titta på statlig tv och propagandafilmer i mer än åtta timmar om dagen. Det säger Navalnyj i sin första intervju med en nyhetsredaktion sedan gripandet i januari. ',
 '"2021-08-25T19:01:48Z"')

In [5]:
doc_example = {}
doc_example["title"], doc_example["article"], doc_example["date"] = example

In [ ]:
# settings.py

In [4]:
ITEM_PIPELINES = ['stack.pipelines.MongoDBPipeline', ]
MONGODB_SERVER = "localhost"
MONGODB_PORT = 27017
MONGODB_DB = "demo"
MONGODB_COLLECTION = "articles"

In [ ]:
class GPCrawler(object):
    name = "gp-crawler"
    allowed_domains = ["gp.se"]
    start_urls = [
        "http://stackoverflow.com/questions?pagesize=50&sort=newest",
    ]

    def parse(self, response):
        questions = Selector(response).xpath('//div[@class="summary"]/h3')

        for question in questions:
            item = StackItem()
            item['title'] = question.xpath(
                'a[@class="question-hyperlink"]/text()').extract()[0]
            item['url'] = question.xpath(
                'a[@class="question-hyperlink"]/@href').extract()[0]
            yield item

In [7]:
import pymongo

In [8]:
connection = pymongo.MongoClient(
    MONGODB_SERVER,
    MONGODB_PORT
)
db = connection[MONGODB_DB]
collection = db[MONGODB_COLLECTION]

In [11]:
db

Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'demo')

In [10]:
collection.insert(dict(doc_example))

/var/folders/5c/9jx1dk7n7z1ggrrgj_9r_jwr0000gn/T/ipykernel_24390/4052541203.py:1: DeprecationWarning: insert is deprecated. Use insert_one or insert_many instead.
  collection.insert(dict(doc_example))


ObjectId('6142621dda0a849b47012611')

In [ ]:
import pymongo

class MongoDBPipeline(object):

    def __init__(self):
        connection = pymongo.MongoClient(
            MONGODB_SERVER,
            MONGODB_PORT
        )
        db = connection[MONGODB_DB]
        self.collection = db[MONGODB_COLLECTION]

    
    
    
    
    def process_item(self, item, spider):
        valid = True
        for data in item:
            if not data:
                valid = False
                raise DropItem("Missing {0}!".format(data))
        if valid:
            self.collection.insert(dict(item))
            log.msg("Question added to MongoDB database!",
                    level=log.DEBUG, spider=spider)
        return item